For simulation competitions Kaggle implements a rating system which calculates deltas based on submissions scores at the moment of play. This, I think, is the main problem of the system. The influence of an episode only takes the history into account and not the future. Additionally, the calculation of deltas and sigmas seems to be unscientific. The Bradley–Terry rating system (BT) solves these drawbacks, a solid maximum-likelihood based method, based on ELO-like probability function and solved with global optimisation.

See info on [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model), and [ELO rating system](https://en.wikipedia.org/wiki/Elo_rating_system#Theory).


Using the data from meta-kaggle dataset, this notebook implements BT scoring, with ELO probability function, and gradient descent optimisation by torch. The output is BT scores for all submissions, and also separately a list of best submissions per team. The association of submission Ids to team names is thanks to the [auxilliary dataset](https://www.kaggle.com/skyramp/luxai2021-submission-team-name-score-over-1500). The notebook can be adopted for other simulation competitions in a straightforward way.

![Lux game screenshot](https://i.imgur.com/OLUObS2.png)

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time
import glob
import collections
import time

import torch as th
from torch import nn

In [ ]:
## You should configure these to your needs. Choose one of ...
# 'hungry-geese', 'rock-paper-scissors', santa-2020', 'halite', 'google-football'
COMP = 'lux-ai-2021'

In [ ]:
ROOT ="."
META = "../input/meta-kaggle/"
COMPETITIONS = {
    'lux-ai-2021': 30067,
    'hungry-geese': 25401,
    'rock-paper-scissors': 22838,
    'santa-2020': 24539,
    'halite': 18011,
    'google-football': 21723
}

Load and filter only our competition.

In [ ]:
# Load Episodes
st = time.time()
episodes_df = pd.read_csv(META + "Episodes.csv", usecols=['CompetitionId','Id'])
print(time.time()-st)
print(f'Episodes.csv: {len(episodes_df)} rows before filtering.')
episodes_df = episodes_df[episodes_df.CompetitionId == COMPETITIONS[COMP]] 
print(f'Episodes.csv: {len(episodes_df)} rows after filtering for {COMP}.')

In [ ]:
# Load EpisodeAgents
st = time.time()
epagents_df = pd.read_csv(META + "EpisodeAgents.csv", usecols=['EpisodeId','Reward','SubmissionId','UpdatedScore'])
print(time.time()-st)
print(f'EpisodeAgents.csv: {len(epagents_df)} rows before filtering.')
epagents_df = epagents_df[epagents_df.EpisodeId.isin(episodes_df.Id)]
print(f'EpisodeAgents.csv: {len(epagents_df)} rows after filtering for {COMP}.')

In [ ]:
epagents_df = epagents_df.sort_values('EpisodeId')

Validate that the even and odd entries are from the same episode

In [ ]:
assert np.all(epagents_df.EpisodeId.values[range(0,len(epagents_df),2)] == epagents_df.EpisodeId.values[range(1,len(epagents_df),2)])

This is what we need from the original dataframe, only submission ids and rewards (to know who won/lost)

In [ ]:
st = time.time()
df = pd.DataFrame(
    np.concatenate(
    [epagents_df.Reward.values.reshape((int(len(epagents_df)/2),2)), 
     epagents_df.SubmissionId.values.reshape((int(len(epagents_df)/2),2)),
    ], axis=1), columns = ['Reward0','Reward1','SubmissionId0','SubmissionId1'])
print(time.time() - st)

Calculate who won and who lost

In [ ]:
df['winner'] = np.where(df.Reward0 > df.Reward1, df.SubmissionId0, df.SubmissionId1)
df['loser'] = np.where(df.Reward0 > df.Reward1, df.SubmissionId1, df.SubmissionId0)

Filter draws. This can be replaced with 0.5 weight for win and 0.5 for lose, but it will require addition of weights.

In [ ]:
df = df.loc[df.Reward0 != df.Reward1]

Filter submissions which do not have at least one win and one lose. The model is not able to handle all-wins or all-loses.

In [ ]:
sz = len(df) + 1
while sz > len(df):
    sz = len(df)
    st = time.time()
    df = df.loc[df.SubmissionId0.isin(df.winner) & df.SubmissionId0.isin(df.loser)]
    df = df.loc[df.SubmissionId1.isin(df.winner) & df.SubmissionId1.isin(df.loser)]
    print(len(df), sz-len(df), time.time() - st)

In [ ]:
items = df.winner.astype(int).unique()
n_items = len(items)

number of submissions

In [ ]:
n_items

number of episodes

In [ ]:
len(df)

Mappins from submission Id to index

In [ ]:
mapping = {s:i for i,s in zip(range(n_items), items)}

Counts and LB scores will be useful later to add to the output dataframes

In [ ]:
cnts = pd.value_counts(df[['SubmissionId0','SubmissionId1']].astype(int).values.reshape(-1))
LB_scores = epagents_df.groupby('SubmissionId').apply(lambda x: x.UpdatedScore.values[-1])

This is the input to the model, which is basically a list of winners and losers for each episode.

In [ ]:
games = np.stack([[mapping[int(s)] for s in df.winner.values],
                  [mapping[int(s)] for s in df.loser.values]], axis=1)

In [ ]:
games.shape

# Model and solving

This is the model implementation, it follows formulas from [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model), and [ELO rating system](https://en.wikipedia.org/wiki/Elo_rating_system#Theory).

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        R = 1000*th.ones((n_items,))
        self.R = nn.Parameter(R.cuda())
        self.idx_i = th.Tensor(games[:,0]).to(int).cuda()
        self.idx_j = th.Tensor(games[:,1]).to(int).cuda()
        
        self.base = th.log(th.Tensor([10]).cuda())
        
    def forward(self):
        
        with th.no_grad():
            self.R -= (self.R.mean() - 1000)
        
        Ri = self.R[self.idx_i]
        Rj = self.R[self.idx_j]
        Qi = th.pow(10, Ri / 400)
        Qj = th.pow(10, Rj / 400)
        
        loss = th.log(Qi + Qj) / self.base - Ri / 400
        
        return loss.mean()

Model and optimizer

In [ ]:
model = Model()

learning_rate = 1e2
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate)

Solving with torch

In [ ]:
for i in range(2000):
    loss = model()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_cpu = loss.item()
    
    if i % 100 == 0:
        print(i, learning_rate, loss_cpu, model.R.min().item(), model.R.grad.abs().max().item())

# Processing the result

Get the model's output and enrich it with auxilliary columns. Calculate the best submission per team by BT score.|

In [ ]:
BT_scores = model.R.cpu().detach().numpy()

info = pd.DataFrame({'BT_score':BT_scores, 'submission':items})
info = info.join(cnts.rename('episodes'), on='submission')
info = info.join(LB_scores.rename('LB_score'), on='submission')
info = info.loc[~info.LB_score.isnull()]

names = pd.read_csv("../input/luxai2021-submission-team-name-score-over-1500/submission_team_names.csv")

info = info.join(names[['SubmissionId','TeamName']].set_index('SubmissionId'), on='submission')
info = info.loc[~info.TeamName.isnull()]
info_best = info.groupby('TeamName').apply(lambda x: x.iloc[np.argmax(x.BT_score.values)])\
    .sort_values('BT_score', ascending=False).reset_index(drop=True)

The best 20 teams, ordered by their best BT-scoring subs.

In [ ]:
info_best.head(20)

Save all submissions and best submissions separately.

In [ ]:
info.to_csv('BT_scores_all')
info_best.to_csv('BT_scores_best')

The END.

![](https://storage.googleapis.com/kaggle-competitions/kaggle/30067/logos/thumb76_76.png?t=2021-07-20-15-37-27)